In [2]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
sequence_length = 28
num_layers = 2
input_size = 28
hidden_size = 128
num_classes = 10
batch_size = 100
learning_rate = 0.01
num_epochs = 5

In [5]:
train_data = torchvision.datasets.MNIST(root="../../data/",
                                       download=True,
                                        train=True,
                                       transform=transforms.ToTensor())
test_data = torchvision.datasets.MNIST(root="../../data/",
                                        train=False,
                                       transform=transforms.ToTensor())

In [6]:
train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                          shuffle=True,
                                          batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                         shuffle=True,
                                         batch_size=batch_size)

In [17]:
# build a BiRNN

class BiRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.input_size = input_size
        self.BiLSTM = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes) # 2 for bidirection
        
    def forward(self, x):
        # Set initial states
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.BiLSTM(x, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        
        return(out)

In [18]:
model = BiRNN(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
# Train Model

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        output = model(images)
        loss = criterion(output, labels)
        
        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.5933
Epoch [1/5], Step [200/600], Loss: 0.2748
Epoch [1/5], Step [300/600], Loss: 0.3085
Epoch [1/5], Step [400/600], Loss: 0.1358
Epoch [1/5], Step [500/600], Loss: 0.1866
Epoch [1/5], Step [600/600], Loss: 0.1896
Epoch [2/5], Step [100/600], Loss: 0.1135
Epoch [2/5], Step [200/600], Loss: 0.1061
Epoch [2/5], Step [300/600], Loss: 0.1286
Epoch [2/5], Step [400/600], Loss: 0.0782
Epoch [2/5], Step [500/600], Loss: 0.0483
Epoch [2/5], Step [600/600], Loss: 0.1402
Epoch [3/5], Step [100/600], Loss: 0.1559
Epoch [3/5], Step [200/600], Loss: 0.0478
Epoch [3/5], Step [300/600], Loss: 0.0537
Epoch [3/5], Step [400/600], Loss: 0.0739
Epoch [3/5], Step [500/600], Loss: 0.0570
Epoch [3/5], Step [600/600], Loss: 0.0987
Epoch [4/5], Step [100/600], Loss: 0.0204
Epoch [4/5], Step [200/600], Loss: 0.0138
Epoch [4/5], Step [300/600], Loss: 0.0516
Epoch [4/5], Step [400/600], Loss: 0.0430
Epoch [4/5], Step [500/600], Loss: 0.0413
Epoch [4/5], Step [600/600], Loss:

In [22]:
# Test model

with torch.no_grad():
    total = correct = 0
    
    for i, (images, labels) in enumerate(test_loader):
        
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum(0).item()
        
print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

Test Accuracy of the model on the 10000 test images: 97.69 %


In [ ]:
torch.save(model.state_dict() ,"BiRNN.ckpt")